## Pydantic Tutorial

[Source](https://docs.pydantic.dev/latest/usage/models/)

* The primary means of defining objects in pydantic is via models (models are simply classes which inherit from BaseModel).

* You can think of models as similar to types in strictly typed languages, or as the requirements of a single endpoint in an API.

* Untrusted data can be passed to a model, and after parsing and validation pydantic guarantees that the fields of the resultant model instance will conform to the field types defined on the model.

### Basic model usage

* `id` is required.
* `name` is not required value but a default value.

In [1]:
from pydantic import BaseModel

class User(BaseModel):
    id: int
    name = 'Peter Cha'

In [2]:
user = User(id='123') # it works! Auto Data Conversion/Casting.
# cause the validation error
# user_x = User(id = '123.45') 

In [3]:
assert user.id == 123
assert isinstance(user.id, int)
assert user.name == 'Peter Cha'
assert user.__fields_set__ == {'id'}
assert user.dict() == dict(user) == {'id': 123, 'name': 'Peter Cha'}

In [4]:
user.dict()

{'id': 123, 'name': 'Peter Cha'}

In [5]:
# This model is mutable.
user.id = 321
user.id

321

### Model properties

* The example above only shows the tip of the iceberg of what models can do. Models possess the following methods and attributes:


`json()`
* returns a JSON string representation dict(); cf. exporting models

`construct()`
* a class method for creating models without running validation; cf. Creating models without validation

`__fields_set__`
* Set of names of fields which were set when the model instance was initialised

`__fields__`
* a dictionary of the model's fields

`__config__`
* the configuration class for the model, cf. model config

In [6]:
user.json()

'{"id": 321, "name": "Peter Cha"}'

In [7]:
user.construct(id = '123.45')

User(id='123.45', name='Peter Cha')

In [8]:
user.__fields_set__

{'id'}

In [9]:
user.__fields__

{'id': ModelField(name='id', type=int, required=True),
 'name': ModelField(name='name', type=str, required=False, default='Peter Cha')}

In [10]:
user.__config__

__main__.Config

In [11]:
user.schema()

{'title': 'User',
 'type': 'object',
 'properties': {'id': {'title': 'Id', 'type': 'integer'},
  'name': {'title': 'Name', 'default': 'Peter Cha', 'type': 'string'}},
 'required': ['id']}

### Recursive Models
* More complex hierarchical data structures can be defined using models themselves as types in annotations.



In [12]:
from typing import Optional
from pydantic import BaseModel

In [16]:
class Foo(BaseModel):
    count: int
    size: Optional[float] = None
        
class Bar(BaseModel):
    apple = 'x'
    banana = 'y'
    
class Spam(BaseModel):
    foo: Foo
    bars: list[Bar]

m = Spam(foo={'count': 4}, 
         bars=[{'apple': 'x1'}, {'apple': 'x2'}])
m

Spam(foo=Foo(count=4, size=None), bars=[Bar(apple='x1', banana='y'), Bar(apple='x2', banana='y')])

In [15]:
m.dict()

{'foo': {'count': 4, 'size': None},
 'bars': [{'apple': 'x1', 'banana': 'y'}, {'apple': 'x2', 'banana': 'y'}]}

### ORM Mode (aka Arbitrary Class Instances)
* Pydantic models can be created from arbitrary class instances to support models that map to ORM(Object Relational Mapping) objects.

To Do this:
1. The Config property `orm_mode` must be set to `True`.
2. The special constructor `from_orm` must be used to createthe model instance.

* In the original tutorial uses SQLAlchemy, but I use MongoDB.
